# UK Yocuda New-Lapsed Shopper Analysis

#### Importing yocuda data from GCS

In [2]:
from pyspark.sql.functions import *

In [6]:
data = (spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("gs://ds-mlengine/praneeth/2017_Yocuda_Dummy_Data_V01_20180129.csv"))\
       .filter((to_date(col("timestamp"))>'2016-11-30') & (to_date(col("timestamp"))<'2017-12-01'))\
  .selectExpr("identifier","to_date(timestamp) as date").distinct()

In [17]:
from pyspark.sql.window import Window

#### Assigning row number

In [21]:
df1=data.withColumn("date2",lead(data["date"]).over(Window.partitionBy("identifier")\
                                                                   .orderBy(desc("date"))))\
        .withColumn("row_num",row_number().over(Window.partitionBy("identifier")\
                                                                  .orderBy(desc("date"))))
df2=df1.filter(col("row_num")==1)
df2.filter(col("date2").isNotNull()).show(5)

+----------+----------+----------+-------+
|identifier|      date|     date2|row_num|
+----------+----------+----------+-------+
| ryd@c.com|2017-08-04|2017-03-31|      1|
| yyj@c.com|2017-02-11|2017-01-11|      1|
| bri@b.com|2017-05-24|2017-01-27|      1|
| bwe@b.com|2017-07-26|2017-07-19|      1|
| iov@c.com|2017-04-28|2017-01-24|      1|
+----------+----------+----------+-------+
only showing top 5 rows



#### Segmentation

In [30]:
df=df2.withColumn("Type of shopper",when(datediff(col("date"),col("date2"))>180,"Lapsed Shopper") \
                                   .when((col("date2").isNull()) | (datediff(col("date"),col("date2"))<=180),"New Shopper"))
df.show(5)

+----------+----------+-----+-------+---------------+
|identifier|      date|date2|row_num|Type of shopper|
+----------+----------+-----+-------+---------------+
| ajb@b.com|2016-12-13| null|      1|    New Shopper|
| bee@f.com|2017-01-05| null|      1|    New Shopper|
| bnx@a.com|2017-02-02| null|      1|    New Shopper|
| buv@e.com|2017-10-01| null|      1|    New Shopper|
| cce@a.com|2017-10-27| null|      1|    New Shopper|
+----------+----------+-----+-------+---------------+
only showing top 5 rows

